In [1]:
import os
import sys
import json
import shutil
import numpy as np
import pandas as pd

In [2]:
# Helper function
def writeProgress(msg, count, total):
    sys.stdout.write(msg + "{:.2%}\r".format(count/total))
    sys.stdout.flush()
    
def newPath(path):
    if not os.path.isdir(path):
        os.mkdir(path)

def read_json(src_path):
    with open(src_path, 'r') as json_file:
        data = json.load(json_file)
    return data

def write_json(data,dst_path):
    with open(dst_path, 'w') as outfile:
        json.dump(data, outfile)

In [3]:
columns = read_json('../../orderedListGenres.json')
print(len(columns), columns)

20 ['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Mystery', 'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western']


In [4]:
# 模型輸出儲存的檔案
# SAVE_NAME = 'resnet50_trainable_8e-5_starwars'
HISTORY_SAVE = './history/' + SAVE_NAME + '.csv'
WEIGHTS_SAVE = './weight/' + SAVE_NAME + '.h5'

NameError: name 'SAVE_NAME' is not defined

# ResNet50 Model

In [5]:
# read dataframe
train_df = pd.read_csv('../filename2genreMat.csv')
test_df = pd.read_csv('../poster2genreMat.csv')
print(train_df.shape, test_df.shape)

(37959, 22) (165, 22)


In [6]:
valid_df = train_df.sample(frac = 0.2, random_state = 42)
train_df = train_df.drop(valid_df.index)
print(train_df.shape, valid_df.shape)

(30367, 22) (7592, 22)


In [7]:
from keras.applications import ResNet50
from keras import models
from keras.models import Model
from keras.optimizers import Adam
from keras_preprocessing.image import ImageDataGenerator
from keras import layers
from keras import regularizers, optimizers
# from keras.applications.model import ResNet152

Using TensorFlow backend.


In [8]:
# 資料路徑
DATASET_PATH  = '../IGimg'

# 影像大小
IMAGE_SIZE = (224, 224)

# 影像類別數
NUM_CLASSES = 20

# 若 GPU 記憶體不足，可調降 batch size 或凍結更多層網路
BATCH_SIZE = 32

# Epoch 數
NUM_EPOCHS = 100

In [9]:
datagen=ImageDataGenerator(rescale=1./255.)
test_datagen=ImageDataGenerator(rescale=1./255.)

train_generator=datagen.flow_from_dataframe(dataframe=train_df,
                                            directory=DATASET_PATH,
                                            x_col="filename",
                                            y_col=columns,
                                            batch_size=BATCH_SIZE,
                                            seed=42,
                                            shuffle=True,
                                            class_mode="other",
                                            target_size=IMAGE_SIZE)

valid_generator=test_datagen.flow_from_dataframe(dataframe=valid_df,
                                                 directory=DATASET_PATH,
                                                 x_col="filename",
                                                 y_col=columns,
                                                 batch_size=BATCH_SIZE,
                                                 seed=42,
                                                 shuffle=True,
                                                 class_mode="other",
                                                 target_size=IMAGE_SIZE)

Found 30367 validated image filenames.
Found 7592 validated image filenames.


# Fixed the last Layer from 1000 classes into N classes

In [ ]:
conv_base = ResNet50(weights='imagenet',
                  include_top=False,
                  input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))
conv_base.summary()

In [ ]:
model = models.Sequential()
model.add(conv_base)        # 將卷積基底視為層加入 Sequential 模型中
model.add(layers.AveragePooling2D(pool_size=(7, 7)))
model.add(layers.Flatten()) # 攤平
model.add(layers.Dense(NUM_CLASSES, activation='sigmoid')) # 增加密集層分類器
model.summary() # 查看模型摘要

In [ ]:
print('This is the number of trainable weights before freezing the conv base:', len(model.trainable_weights))

conv_base.trainable = True

print('This is the number of trainable weights after freezing the conv base:', len(model.trainable_weights))

In [ ]:
model.compile(optimizer=Adam(lr=8e-5),
              loss='binary_crossentropy',
              metrics=['acc'])
model.summary()

# Training Model

In [ ]:
STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n // valid_generator.batch_size

In [ ]:
# 訓練模型
from keras.callbacks import EarlyStopping
callbacks = [EarlyStopping(monitor='val_loss')]

history = model.fit_generator(generator = train_generator,
                              steps_per_epoch = STEP_SIZE_TRAIN,
                              validation_data = valid_generator,
                              validation_steps = STEP_SIZE_VALID,
                              epochs = NUM_EPOCHS,
                              callbacks=callbacks)

In [ ]:
# 儲存訓練好的模型
model.save(WEIGHTS_SAVE)

In [ ]:
# save history
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

his_df = pd.DataFrame(data = {
    'Epoch' : epochs,
    'Loss' : loss,
    'Acc' : acc,
    'Val_loss' : val_loss,
    'Val_acc' : val_acc
})
his_df = his_df[['Epoch', 'Loss', 'Acc', 'Val_loss', 'Val_acc']]
his_df.to_csv(HISTORY_SAVE, index = False)

# Plot acc & loss

In [ ]:
# 繪製結果
import matplotlib.pyplot as plt

plt.figure()

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.show()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

# Testing

### Testing set: Poster

In [10]:
SAVE_NAME = SAVE_NAME + '_poster'

NameError: name 'SAVE_NAME' is not defined

In [11]:
test_df = pd.read_csv('../poster2genreMat.csv')
test_df

,from,filename,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,...,History,Horror,Music,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,../Profiles_165/21bridgesmovie/poster.jpeg,21bridgesmovie.jpg,1,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,../Profiles_165/47metersdown/poster.jpeg,47metersdown.jpg,0,1,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
2,../Profiles_165/abeautifuldaymovie/poster.jpeg,abeautifuldaymovie.jpg,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,../Profiles_165/abominablemovie/poster.jpeg,abominablemovie.jpg,0,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,../Profiles_165/adastramovie/poster.jpeg,adastramovie.jpg,0,1,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,../Profiles_165/wrinklestheclown/poster.jpeg,wrinklestheclown.jpg,0,0,0,0,1,0,1,0,...,0,1,0,0,0,0,0,0,0,0
161,../Profiles_165/xmenmovies/poster.jpeg,xmenmovies.jpg,1,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
162,../Profiles_165/yardiefilm/poster.jpeg,yardiefilm.jpg,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
163,../Profiles_165/yesterdaymovie/poster.jpeg,yesterdaymovie.jpg,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [12]:
DATASET_PATH = '../Poster'

In [13]:
test_generator=test_datagen.flow_from_dataframe(dataframe=test_df,
                                                directory=DATASET_PATH,
                                                x_col="filename",
                                                batch_size=1,
                                                seed=42,
                                                shuffle=False,
                                                class_mode=None,
                                                target_size=IMAGE_SIZE)

Found 165 validated image filenames.


In [14]:
STEP_SIZE_TEST = test_generator.n // test_generator.batch_size

In [15]:
test_generator.reset()
pred=model.predict_generator(test_generator,
                             steps=STEP_SIZE_TEST,
                             verbose=1)

NameError: name 'model' is not defined

# Output

In [ ]:
pred.shape

In [ ]:
predictions = pred

results=pd.DataFrame(predictions, columns=columns)
results["filename"] = test_generator.filenames
ordered_cols = ["filename"] + columns
results = results[ordered_cols] #To get the same column order
results.to_csv("./output/"+ SAVE_NAME + "_result.csv",index=False)

## Label top n dynamic

In [ ]:
true_df = test_df

In [ ]:
results = pd.read_csv("./output/"+ SAVE_NAME + "_result.csv")
results

In [ ]:
labels = read_json('../../genresDic.json')

In [ ]:
def countAcc(predList, trueList):
    fraction = 0
    denominator = len(trueList)
    for g in predList:
        if g in trueList:
            fraction += 1 
    return fraction / denominator

In [ ]:
li = []
for i, row in true_df.iterrows():
    username = row['filename']
    if username == results.loc[i,:]['filename']: # username equals
        print(i, username)
        accname = username.split('.')[0]
        truth = labels[accname]
        numTrue = sum(row[2:])
        print('count:', numTrue)
        
        s = results.loc[i,:][1:]
        top_n = s.sort_values(ascending=False)[:numTrue]
        inList = list(top_n.index)
        
        onehot = []
        for g in columns:
            onehot.append(int(g in inList))
            
        accu = countAcc(inList, truth)
        print(accu, inList, truth)
        
        appList = [username, accu, truth, inList] + onehot
        li.append(appList)
        

In [ ]:
acc_df = pd.DataFrame(li, columns = ['username', 'acc', 'truth', 'top_n'] + columns)
acc_df

In [ ]:
acc_df.to_csv("./output/"+ SAVE_NAME + "_bi.csv", index = 0)

In [ ]:
acc_df['acc'].mean()

# Feature Extraction
## 測試reload model準確度

In [16]:
SAVE_NAME = 'test'
DATASET_PATH = '../Poster'

In [17]:
WEIGHTS_LOAD = './weight/resnet50_trainable_9e-5_starwars.h5'

from keras.models import load_model

model = load_model(WEIGHTS_LOAD)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [31]:
# reload weights
NAMES = os.listdir('./weight/')
print(NAMES)

['resnet50_trainable_2e_alltraining.h5', 'resnet50_trainable_5e-4_alltraining.h5', 'resnet50_trainable_8e-5_starwars.h5', 'resnet50_trainable_9e-5_alltraining.h5', 'resnet50_trainable_9e-5_starwars.h5', 'resnet50_trainable_2e_100training.h5', 'resnet50_trainable_9e-5_newdata.h5', 'resnet50_trainable_1e-4_alltraining.h5', 'resnet50_trainable_5e-5_starwars.h5', 'resnet50_trainable_9e_alltraining.h5', 'resnet50.h5', 'resnet50_trainable_7e-5_starwars.h5', 'resnet50_trainable_3e-5_starwars.h5', 'resnet50_trainable_1e-4_starwars.h5', 'resnet50_trainable_3e-4_alltraining.h5', 'resnet50_trainable_9e-5_alltraining_.h5', 'resnet50_trainable_1e_alltraining.h5', 'resnet50_trainable_5e_alltraining.h5', 'resnet50_trainable_7e_alltraining.h5', 'resnet50_trainable_2e_500training.h5', 'resnet50_trainable_3e_alltraining.h5', 'resnet50_trainable_2e.h5']


In [34]:
# reload weights
for NAME in NAMES:
    print(NAME)
    WEIGHTS_LOAD = './weight/' + NAME
    model = load_model(WEIGHTS_LOAD)
    test_generator=test_datagen.flow_from_dataframe(dataframe=test_df,
                                                    directory=DATASET_PATH,
                                                    x_col="filename",
                                                    batch_size=1,
                                                    seed=42,
                                                    shuffle=False,
                                                    class_mode=None,
                                                    target_size=IMAGE_SIZE)
    
    STEP_SIZE_TEST = test_generator.n // test_generator.batch_size
    test_generator.reset()
    pred=model.predict_generator(test_generator,
                                 steps=STEP_SIZE_TEST,
                                 verbose=1)
    
    predictions = pred

    results=pd.DataFrame(predictions, columns=columns)
    results["filename"] = test_generator.filenames
    ordered_cols = ["filename"] + columns
    results = results[ordered_cols] #To get the same column order
    # results.to_csv("./output/"+ SAVE_NAME + "_result.csv",index=False)

    li = []
    for i, row in true_df.iterrows():
        username = row['filename']
        if username == results.loc[i,:]['filename']: # username equals
#             print(i, username)
            accname = username.split('.')[0]
            truth = labels[accname]
            numTrue = sum(row[2:])
#             print('count:', numTrue)

            s = results.loc[i,:][1:]
            top_n = s.sort_values(ascending=False)[:numTrue]
            inList = list(top_n.index)

            onehot = []
            for g in columns:
                onehot.append(int(g in inList))

            accu = countAcc(inList, truth)
#             print(accu, inList, truth)

            appList = [username, accu, truth, inList] + onehot
            li.append(appList)
            
    acc_df = pd.DataFrame(li, columns = ['username', 'acc', 'truth', 'top_n'] + columns)
    print(acc_df['acc'].mean())

resnet50_trainable_2e_alltraining.h5
Found 165 validated image filenames.
165/165 [==============================] - 15s 92ms/step
0.5040404040404041
resnet50_trainable_5e-4_alltraining.h5
Found 165 validated image filenames.
165/165 [==============================] - 18s 108ms/step
0.47979797979797983
resnet50_trainable_8e-5_starwars.h5
Found 165 validated image filenames.
165/165 [==============================] - 21s 128ms/step
0.6939393939393942
resnet50_trainable_9e-5_alltraining.h5
Found 165 validated image filenames.
165/165 [==============================] - 24s 145ms/step
0.6555555555555553
resnet50_trainable_9e-5_starwars.h5
Found 165 validated image filenames.
165/165 [==============================] - 29s 176ms/step
0.7181818181818185
resnet50_trainable_2e_100training.h5
Found 165 validated image filenames.
165/165 [==============================] - 33s 201ms/step
0.3797979797979798
resnet50_trainable_9e-5_newdata.h5
Found 165 validated image filenames.
165/165 [===========

/home/tonylab/miniconda3/envs/tf/lib/python3.6/site-packages/keras/engine/saving.py:384: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


Found 165 validated image filenames.
165/165 [==============================] - 61s 368ms/step
0.15252525252525248
resnet50_trainable_7e-5_starwars.h5
Found 165 validated image filenames.
165/165 [==============================] - 69s 420ms/step
0.7111111111111115
resnet50_trainable_3e-5_starwars.h5
Found 165 validated image filenames.
165/165 [==============================] - 78s 474ms/step
0.6343434343434344
resnet50_trainable_1e-4_starwars.h5
Found 165 validated image filenames.
165/165 [==============================] - 94s 569ms/step
0.6969696969696971
resnet50_trainable_3e-4_alltraining.h5
Found 165 validated image filenames.
165/165 [==============================] - 106s 643ms/step
0.428282828282828
resnet50_trainable_9e-5_alltraining_.h5
Found 165 validated image filenames.
165/165 [==============================] - 121s 735ms/step
0.5737373737373738
resnet50_trainable_1e_alltraining.h5
Found 165 validated image filenames.
165/165 [==============================] - 133s 807ms

In [18]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
average_pooling2d_1 (Average (None, 1, 1, 2048)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                40980     
Total params: 23,628,692
Trainable params: 23,575,572
Non-trainable params: 53,120
_________________________________________________________________


In [19]:
test_df = pd.read_csv('../poster2genreMat.csv')
test_df

,from,filename,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,...,History,Horror,Music,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,../Profiles_165/21bridgesmovie/poster.jpeg,21bridgesmovie.jpg,1,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,../Profiles_165/47metersdown/poster.jpeg,47metersdown.jpg,0,1,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
2,../Profiles_165/abeautifuldaymovie/poster.jpeg,abeautifuldaymovie.jpg,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,../Profiles_165/abominablemovie/poster.jpeg,abominablemovie.jpg,0,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,../Profiles_165/adastramovie/poster.jpeg,adastramovie.jpg,0,1,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,../Profiles_165/wrinklestheclown/poster.jpeg,wrinklestheclown.jpg,0,0,0,0,1,0,1,0,...,0,1,0,0,0,0,0,0,0,0
161,../Profiles_165/xmenmovies/poster.jpeg,xmenmovies.jpg,1,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
162,../Profiles_165/yardiefilm/poster.jpeg,yardiefilm.jpg,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
163,../Profiles_165/yesterdaymovie/poster.jpeg,yesterdaymovie.jpg,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [20]:
test_generator=test_datagen.flow_from_dataframe(dataframe=test_df,
                                                directory=DATASET_PATH,
                                                x_col="filename",
                                                batch_size=1,
                                                seed=42,
                                                shuffle=False,
                                                class_mode=None,
                                                target_size=IMAGE_SIZE)

Found 165 validated image filenames.


In [21]:
STEP_SIZE_TEST = test_generator.n // test_generator.batch_size

In [22]:
test_generator.reset()
pred=model.predict_generator(test_generator,
                             steps=STEP_SIZE_TEST,
                             verbose=1)

165/165 [==============================] - 15s 88ms/step


In [23]:
pred.shape

(165, 20)

In [25]:
predictions = pred

results=pd.DataFrame(predictions, columns=columns)
results["filename"] = test_generator.filenames
ordered_cols = ["filename"] + columns
results = results[ordered_cols] #To get the same column order
# results.to_csv("./output/"+ SAVE_NAME + "_result.csv",index=False)

In [26]:
true_df = test_df
labels = read_json('../../genresDic.json')

In [27]:
def countAcc(predList, trueList):
    fraction = 0
    denominator = len(trueList)
    for g in predList:
        if g in trueList:
            fraction += 1 
    return fraction / denominator

In [28]:
li = []
for i, row in true_df.iterrows():
    username = row['filename']
    if username == results.loc[i,:]['filename']: # username equals
        print(i, username)
        accname = username.split('.')[0]
        truth = labels[accname]
        numTrue = sum(row[2:])
        print('count:', numTrue)
        
        s = results.loc[i,:][1:]
        top_n = s.sort_values(ascending=False)[:numTrue]
        inList = list(top_n.index)
        
        onehot = []
        for g in columns:
            onehot.append(int(g in inList))
            
        accu = countAcc(inList, truth)
        print(accu, inList, truth)
        
        appList = [username, accu, truth, inList] + onehot
        li.append(appList)
        

0 21bridgesmovie.jpg
count: 3
0.6666666666666666 ['Action', 'Crime', 'Sci-Fi'] ['Action', 'Crime', 'Drama']
1 47metersdown.jpg
count: 3
0.6666666666666666 ['Action', 'Adventure', 'Horror'] ['Adventure', 'Drama', 'Horror']
2 abeautifuldaymovie.jpg
count: 2
1.0 ['Drama', 'Biography'] ['Biography', 'Drama']
3 abominablemovie.jpg
count: 3
1.0 ['Animation', 'Comedy', 'Adventure'] ['Adventure', 'Animation', 'Comedy']
4 adastramovie.jpg
count: 3
0.6666666666666666 ['Drama', 'Fantasy', 'Mystery'] ['Adventure', 'Drama', 'Mystery']
5 adogsjourneymovie.jpg
count: 3
1.0 ['Drama', 'Comedy', 'Adventure'] ['Adventure', 'Comedy', 'Drama']
6 aftermathmovie.jpg
count: 3
0.6666666666666666 ['Romance', 'Drama', 'Biography'] ['Drama', 'Romance', 'War']
7 aftermovie.jpg
count: 2
0.5 ['Romance', 'Horror'] ['Drama', 'Romance']
8 ahiddenlifefilm.jpg
count: 3
0.0 ['Thriller', 'Mystery', 'Horror'] ['Biography', 'Drama', 'War']
9 alitamovie.jpg
count: 3
1.0 ['Action', 'Adventure', 'Sci-Fi'] ['Action', 'Adventure'

0.3333333333333333 ['Drama', 'Comedy', 'Thriller'] ['Action', 'Crime', 'Drama']
144 thelegomovie.jpg
count: 3
1.0 ['Action', 'Adventure', 'Animation'] ['Action', 'Adventure', 'Animation']
145 themustangfilm.jpg
count: 1
1.0 ['Drama'] ['Drama']
146 thenightingalefilm.jpg
count: 3
0.6666666666666666 ['Mystery', 'Thriller', 'Drama'] ['Adventure', 'Drama', 'Thriller']
147 theprodigymovie.jpg
count: 2
0.5 ['Drama', 'Thriller'] ['Horror', 'Thriller']
148 thesunisalsoastar.jpg
count: 2
1.0 ['Drama', 'Romance'] ['Drama', 'Romance']
149 theupsidefilm.jpg
count: 2
1.0 ['Comedy', 'Drama'] ['Comedy', 'Drama']
150 tolkienmovie.jpg
count: 3
0.6666666666666666 ['Drama', 'Biography', 'History'] ['Biography', 'Drama', 'War']
151 toystory.jpg
count: 3
1.0 ['Comedy', 'Animation', 'Adventure'] ['Adventure', 'Animation', 'Comedy']
152 triplefrontier.jpg
count: 3
0.6666666666666666 ['Drama', 'Action', 'Crime'] ['Action', 'Adventure', 'Crime']
153 uglydolls.jpg
count: 3
1.0 ['Comedy', 'Adventure', 'Animation

In [29]:
acc_df = pd.DataFrame(li, columns = ['username', 'acc', 'truth', 'top_n'] + columns)
acc_df['acc'].mean()

0.7181818181818185

## 萃取97部電影imdb的feature

### Poster Feature Vector

In [ ]:
test_generator

In [ ]:
extract_layer = Model(inputs=model.input,
                      outputs=model.get_layer('flatten_1').output)
#以这个model的预测值作为输出
extract_output = extract_layer.predict_generator(test_generator,
                                                 steps=STEP_SIZE_TEST,
                                                 verbose=1)
print(extract_output.shape)

In [ ]:
idx = test_generator.filenames
print(len(idx), idx)

In [ ]:
res=pd.DataFrame(extract_output, index = idx)
res

In [ ]:
res.to_csv("./output/PosterFeatureVec.csv", header=False)

### IG images

In [ ]:
DATASET_PATH  = '../IGimg'
train_df = pd.read_csv('../filename2genreMat.csv')
train_df

In [ ]:
train_generator=datagen.flow_from_dataframe(dataframe=train_df,
                                            directory=DATASET_PATH,
                                            x_col="filename",
                                            batch_size=1,
                                            seed=42,
                                            shuffle=False,
                                            class_mode=None,
                                            target_size=IMAGE_SIZE)

In [ ]:
STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
print(STEP_SIZE_TRAIN)

In [ ]:
extract_layer = Model(inputs=model.input,
                      outputs=model.get_layer('flatten_1').output)
#以这个model的预测值作为输出
extract_output = extract_layer.predict_generator(train_generator,
                                                 steps=STEP_SIZE_TRAIN,
                                                 verbose=1)
print(extract_output.shape)

In [ ]:
idx = train_generator.filenames
print(len(idx), idx)

In [ ]:
res=pd.DataFrame(extract_output, index = idx)
res

In [ ]:
res.to_csv("./output/IGimgFeatureVec.csv", header=False)